Mám 75 studentů a 11 je leváků.\
Splňuje tato skupina lidí předpoklad, že 12 % lidí je leváků?\
int(0.12*75) = 9\
\
Teď si udělám tabulku:

    Typ     Exp   Teor
    Levák   11      9
    Pravák  64      66
    Celkem  75      75 

H0: p == 12% \
HA: p != 12% \
Významnost: 5%

Teď potřebuji stupně volnosti, použiji 1, protože na to, abych zjistil, kolik je leváků, mi stačí odečíst od celkového množství jeden údaj: počet praváků

\begin{align*}
\chi^2_{DF=1} =  \sum_{i=1}^n{\frac{(e_i - t_i)^2}{t_i}}
\end{align*}

ei ... experimentální hodnota\
ti ... teoretická hodnota

\
Podmínkou testu je, že frekvence četnosti TEORETICKÝCH (OČEKÁVANÝCH) hodnot jsou všechny větší než 5.
- 9 > 5
- 66 > 5
- OK

\begin{align*}
\chi^2_{DF=1} =  \sum_{i=1}^n{\frac{(e_i - t_i)^2}{t_i}} = \frac{(11 - 9)^2}{9} + \frac{(64 - 66)^2}{66} \newline
\chi^2_{DF=1} =  50.5\%
\end{align*}

Jak rozhodnu (zamítnu?)\
Rozhodl jsem se pro 5% významnost.\
PDF(Chi kvadrát) vypadá takto\
<img src="./picture_chisquaredTestGOF.PNG" alt="Chi-Squared Test Goodness of Fit 1DOF"  width="300"  height="200" /> \
Já potřebuji, aby hodnota byla níže než na 5% od konce, tedy 1- 0.05\
A pro test hypotézy si pomůžu inverzní funkcí její kumulace CDF, tedy percent point function PPF - třeba ze scipy.stats, nebo =CHISQ.INV(1-CONFIDENCELEVEL, DOF) v Excelu\

Hodnoty ze software: ChiKvadrat(DOF=1) > 3.841 zamítnu\
0.505 < 3.841: NEZAMÍTNU NULOVOU HYPOTÉZU\

NENÍ DOST DŮKAZŮ, ŽE POČET LEVÁKŮ JE JINÝ NEŽ 12 % \


Jinou možností je použít p-Value test:\
p < 0.05\
K tomu využiji hodnotu z kumulativní distribuční funkce CFD od Chi-kvadrátu pro spočítanou chi-kvadrát hodnotu.\
Například v Excelu: IF ( 1 - CHISQ.DIST(CHISQ, DOF, TRUE), True, False)\
Zde v příkladě  IF ( 1 - CHISQ.DIST(0.505, 1, True), True, False) = 0.477\
<img src="./picture_chisquaredTestGOF2.PNG" alt="Chi-Squared Test Goodness of Fit 1DOF"  width="300"  height="200" /> \
\
\Proč 1 - ... ? Chci hodnotu napravo od 0.505, kdybych chtěl nalevo, tak to bude jen to CHISQ.DIST...
pValue: 0.477 > 0.05: neplatí - nemohu zamítnout nultou hypotézu




\
\
\


Příklad: 
Majitel obchodu tvrdí, že mu chodí procentáž zákazníků do obchodu.\
Já mu nevěřím, a tak jsem si jeho distribuci ověřil.\

    Den Po Út  St  Čt  Pá  So
    %   10  10  15  20  30  15
    Já  30  14  34  45  57  20

Předpokládám významnost 5 %.\

In [11]:
from typing import Sequence
from numbers import Number
from scipy.stats import chi2
def ChiSquareStatistic(observed: Number, expected: Number) -> float:
    numerator: Number   = (observed - expected)**2
    denominator: Number = expected
    result: float       = numerator/denominator
    return result

def getDOF(rows: int, columns: int) -> int:
    #TIP: To get the degrees of freedom, we use the following formula:
    #(r-1)(c-1) where r= the number of rows and c= the number of columns!
    return (rows-1)*(columns-1)
    
if __name__ == "__main__":
    dailyPercentages: Sequence[float]   = 0.1, 0.1, 0.15, 0.2, 0.3, 0.15
    sanityTest: bool                    = sum(dailyPercentages) == 1

    if sanityTest:
        significance: float                 = 0.05
        
        rows: int                           = 2
        columns: int                        = 6
        DOF: int                            = getDOF(rows, columns) #nicmene mam 6 hodnot, takze jen 5 muzu menit, 6 dopocitam -> DOF
        
        
        dailyObserved_me: Sequence[int]     = 30, 14, 34, 45, 57, 20
        totalObserved_me: int               = sum(dailyObserved_me)
        dailyObserved_him: Sequence[int]    = [int(percentage*totalObserved_me) for percentage in dailyPercentages]
        print("Him: ", dailyObserved_him)
        print("Me:  ", dailyObserved_me)
        
        chiSquares: list[float]             = [ChiSquareStatistic(obs, expctd) for obs, expctd in zip (dailyObserved_me, dailyObserved_him)]
        print("Chi-squares: ", [f"{val:.3F}" for val in chiSquares])
        
        chiSquareSum: float                 = sum(chiSquares)
        print(f"Chi-square sum: {chiSquareSum:.3F}")
        
        confidence: float                   = 1. - significance # zajímá mne pravý ocásek, takže důvěrnost
        chiSquareCrit: float                = chi2.ppf(confidence, DOF)
        print(f"Chi-square_CRIT<{significance}>_{DOF} = {chiSquareCrit}")
        
        passed: bool                        = chiSquareSum < chiSquareCrit
        if passed:  print(f"The shopkeeper is correct, as the chiSquareSum < chiSquareCrit... {chiSquareSum} > {chiSquareCrit}")
        else:       print(f"The shopkeeper is not correct, as the chiSquareSum > chiSquareCrit... {chiSquareSum} > {chiSquareCrit}")
        
    else:
        print("Shop keeper gave a result which does not add up to 100 %")



Him:  [20, 20, 30, 40, 60, 30]
Me:   (30, 14, 34, 45, 57, 20)
Chi-squares:  ['5.000', '1.800', '0.533', '0.625', '0.150', '3.333']
Chi-square sum: 11.442
Chi-square_CRIT<0.05>_5 = 11.070497693516351
The shopkeeper is not correct, as the chiSquareSum > chiSquareCrit... 11.441666666666666 > 11.070497693516351


\
\
\

Příklad: \
Petra zjistila, že minulý rok na kliniku chodili pacienti a čekali průměrně minut dle tabulky níže:

    t   15  24  37  60
    %   50  30  10  10

Rozhodla se ověřit pomocí chi-kvadrát testu goodness of fit, jestli i tento rok to platí. Napočítala:

    t   15  24  37  60
    n   75  55  15  5

Jaká je pValue? [zde použijeme namísto PPF... 1 - CDF]

In [19]:
from typing import Sequence
from numbers import Number
from scipy.stats import chi2
def ChiSquareStatistic(observed: Number, expected: Number) -> float:
    numerator: Number   = (observed - expected)**2
    denominator: Number = expected
    result: float       = numerator/denominator
    return result

def getDOF(rows: int, columns: int) -> int:
    #TIP: To get the degrees of freedom, we use the following formula:
    #(r-1)(c-1) where r= the number of rows and c= the number of columns!
    return (rows-1)*(columns-1)
    
if __name__ == "__main__":
    dailyPercentages: Sequence[float]   = 0.5, 0.3, 0.1, 0.1
    sanityTest: bool                    = sum(dailyPercentages) == 1

    if sanityTest:      
        rows: int                           = 2
        columns: int                        = len(dailyPercentages)
        DOF: int                            = getDOF(rows, columns) #nicmene mam 6 hodnot, takze jen 5 muzu menit, 6 dopocitam -> DOF
        
        
        dailyObserved_me: Sequence[int]     = 75, 55, 15, 5
        totalObserved_me: int               = sum(dailyObserved_me)
        dailyObserved_him: Sequence[int]    = [int(percentage*totalObserved_me) for percentage in dailyPercentages]
        print("Him: ", dailyObserved_him)
        print("Me:  ", dailyObserved_me)
        if any(value < 5 for value in dailyObserved_him):
            print("The test will fail anyways")
        
        else:
            chiSquares: list[float]             = [ChiSquareStatistic(obs, expctd) for obs, expctd in zip (dailyObserved_me, dailyObserved_him)]
            print("Chi-squares: ", [f"{val:.3F}" for val in chiSquares])
            
            chiSquareSum: float                 = sum(chiSquares)
            print(f"Chi-square sum: {chiSquareSum:.3F}")
            
            pValue: float                       = 1 - chi2.cdf(chiSquareSum, DOF)
            print(f"pValue<{chiSquareSum:.3F}>_{DOF} = {pValue:.3F}")
    else:
        print("Shop keeper gave a result which does not add up to 100 %")



Him:  [75, 45, 15, 15]
Me:   (75, 55, 15, 5)
Chi-squares:  ['0.000', '2.222', '0.000', '6.667']
Chi-square sum: 8.889
pValue<8.889>_3 = 0.031


Příklad\
Les je rozdělen do 4 regionů a lesník chtěl zjistit, jestli počet jelenů v každém koresponduje s jejich rozlohou.

    reg[m2] A   B   C   D
    S[%]    38  30  31  1
    viděl   408 329 330 3
    čekal   407 321 332 11

\begin{align*}
    \chi^2=5.75 \newline
    \text{DF}=3 \newline
    p = 0.124 \newline
    \alpha = 0.1
\end{align*}

H0: počet jelenů koresponduje s rozlohou regionů\
HA: počet jelenů nekoresponduje s rozlohou regionů

\begin{aligned}
&\text{P-value }< \alpha \rightarrow \text{reject } \text{H}_0 \rightarrow \text{accept } \text{H}_\text{a} \\
&\text{P-value } \geq \alpha \rightarrow \text{fail to reject } \text{H}_0 \rightarrow \text{cannot accept } \text{H}_\text{a}
\end{aligned}

pValue < alpha: zamítnu nulovou hypotézu a přijmu alternativu\
pValue >= alpha: zamítnu alternativní hypotézu\

0.124 >= 1:
- zamítnul jsem nulovou hypotézu
- ale taky nemohu přijmout alternativní

